In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

# 分析00

> 00_Gen によって生成されたデータに対して各種モデルを適用する

In [ ]:
#| hide
from typing import NamedTuple
from KalmanPaper import gen00, simple as sp
import jax.random as jrd
import jax.numpy as jnp
import jax.random as jrd
import jax
from jaxtyping import Float, Int, Array, PRNGKeyArray
from KalmanPaper import gen00, EKF, VA, simple
from matplotlib import pyplot as plt
from typing import Tuple, Callable
from functools import partial
import h5py

## 関数定義

In [ ]:
#| hide
class Parameter(NamedTuple):
  r"""$\!$*
  00_Gen によって生成するときのパラメータ。

  table
  G: $\boldsymbol\Gamma$
  S: $\boldsymbol\Sigma$
  w0: $\mathbf w_{-1}$
  P0: $\mathbf P_{-1}$
  epsilon: $\epsilon$

  *$\!$"""

  G: Float[Array, "{N} {N}"]
  S: Float[Array, "{N} {N}"]
  w0: Float[Array, "{N}"]
  P0: Float[Array, "{N} {N}"]
  epsilon: Float[Array, ""] = jnp.array(2**-16)

sp.rewrite_nt(Parameter)

In [ ]:
#| echo: false
sp.reshow_doc(Parameter)

---

### Parameter

>      Parameter (G:jaxtyping.Float[Array,'{N}{N}'],
>                 S:jaxtyping.Float[Array,'{N}{N}'],
>                 w0:jaxtyping.Float[Array,'{N}'],
>                 P0:jaxtyping.Float[Array,'{N}{N}'],
>                 epsilon:jaxtyping.Float[Array,'']=Array(1.5258789e-05,
>                 dtype=float32, weak_type=True))

*$\!$*
  00_Gen によって生成するときのパラメータ。

  | $\!$ | Type | Default | Details |
|--|--|--|--|
  | G | Float[Array, '{N} {N}'] | $\!$ | $\boldsymbol\Gamma$
  | S | Float[Array, '{N} {N}'] | $\!$ | $\boldsymbol\Sigma$
  | w0 | Float[Array, '{N}'] | $\!$ | $\mathbf w_{-1}$
  | P0 | Float[Array, '{N} {N}'] | $\!$ | $\mathbf P_{-1}$
  | epsilon | Float[Array, ''] | 1.52587890625e-05 | $\epsilon$

  *$\!$*

In [ ]:
#| hide
class Param(NamedTuple):
  r"""$\!$*
  可変パラメータ。各パラメータは次のように定義される。

  - $N$
  - $T=1000$
  - $\boldsymbol\Gamma=2^p\mathbf I$
  - $\boldsymbol\Sigma=2^q\mathbf I$
  - $\mathbf w_{-1}=(r/2)(1,\ldots,1)^T/\sqrt{N}$
  - $\mathbf P_{-1}=\boldsymbol\Gamma$
  - $\epsilon=2^{-16}$

  table
  N: $N$
  p: $p$
  q: $q$
  r: $r$

  *$\!$"""
  N: int
  p: int
  q: int
  r: int

sp.rewrite_nt(Param)

In [ ]:
#| echo: false
sp.reshow_doc(Param)

---

### Param

>      Param (N:int, p:int, q:int, r:int)

*$\!$*
  可変パラメータ。各パラメータは次のように定義される。

  - $N$
  - $T=1000$
  - $\boldsymbol\Gamma=2^p\mathbf I$
  - $\boldsymbol\Sigma=2^q\mathbf I$
  - $\mathbf w_{-1}=(r/2)(1,\ldots,1)^T/\sqrt{N}$
  - $\mathbf P_{-1}=\boldsymbol\Gamma$
  - $\epsilon=2^{-16}$

  | $\!$ | Type | Details |
|--|--|--|
  | N | int | $N$
  | p | int | $p$
  | q | int | $q$
  | r | int | $r$

  *$\!$*

In [ ]:
#| hide
def restore_param(
    param: Param # Param
) -> Tuple[int, int, Parameter]: # $N$, $T$, Parameter
  r"""$\!$*
  Param から Parameter に変換する。
  *$\!$"""
  return param.N, 1000, Parameter(
    2.0**param.p * jnp.identity(param.N),
    2.0**param.q * jnp.identity(param.N),
    (param.r/(2.0*jnp.sqrt(param.N)))*jnp.ones(param.N),
    2.0**param.p * jnp.identity(param.N),
  )

In [ ]:
#| echo: false
sp.reshow_doc(restore_param)

---

### restore_param

>      restore_param (param:__main__.Param)

*$\!$*
Param から Parameter に変換する。
*$\!$*

|$\!$| **Type** | **Details** |
| -- | -------- | ----------- |
| param | Param | Param |
| **Returns** | **Tuple** | **$N$, $T$, Parameter** |

In [ ]:
#| hide
def save_data(
    param: Param, # Param
    name: str, # Model name (Gen, EKF, etc.)
    data: dict, # dataset_name: jnp.array
) -> None:
  r"""$\!$*
  `00_Data.h5` にデータを格納する

  ```
.
├───param1
│   ├───Gen
│   │       W (seed, T, N)
│   │       X (seed, T, N)
│   │       Y (seed, T)
│   │
│   ├───Model1
│   │       W (seed, T, N)
│   │       P (seed, T, N, N)
│   │
│   ├───Model2
│
├───param2
```

  *$\!$"""

  with h5py.File("00_Data.h5", mode="a") as f:
    dirparam = f"/p{param.N}{param.p}{param.q}{param.r}"
    groupparam = f.require_group(name=dirparam)
    paramdict = param._asdict()
    for k, v in paramdict.items():
      groupparam.attrs[k] = v
    groupmodel = groupparam.require_group(name=name)
    for k, v in data.items():
      try:
        groupmodel.create_dataset(name=k, data=v, compression="gzip")
      except ValueError as e:
        print(e)

In [ ]:
#| echo: false
sp.reshow_doc(save_data)

---

### save_data

>      save_data (param:__main__.Param, name:str, data:dict)

*$\!$*
  `00_Data.h5` にデータを格納する

  ```
.
├───param1
│   ├───Gen
│   │       W (seed, T, N)
│   │       X (seed, T, N)
│   │       Y (seed, T)
│   │
│   ├───Model1
│   │       W (seed, T, N)
│   │       P (seed, T, N, N)
│   │
│   ├───Model2
│
├───param2
```

  *$\!$*

|$\!$| **Type** | **Details** |
| -- | -------- | ----------- |
| param | Param | Param |
| name | str | Model name (Gen, EKF, etc.) |
| data | dict | dataset_name: jnp.array |
| **Returns** | **None** |$\!$|

In [ ]:
#| hide
class WXY(NamedTuple):
  r"""$\!$*
  
  table
  W: $\{\mathbf w_t\}_{t=0,\ldots,T-1}$
  X: $\{\mathbf x_t\}_{t=0,\ldots,T-1}$
  Y: $\{y_t\}_{t=0,\ldots,T-1}$

  *$\!$"""
  W: Float[Array, "T N"]
  X: Float[Array, "T N"]
  Y: Float[Array, "T"]

sp.rewrite_nt(WXY)

In [ ]:
#| echo: false
sp.reshow_doc(WXY)

---

### WXY

>      WXY (W:jaxtyping.Float[Array,'TN'], X:jaxtyping.Float[Array,'TN'],
>           Y:jaxtyping.Float[Array,'T'])

*$\!$*

  | $\!$ | Type | Details |
|--|--|--|
  | W | Float[Array, 'T N'] | $\{\mathbf w_t\}_{t=0,\ldots,T-1}$
  | X | Float[Array, 'T N'] | $\{\mathbf x_t\}_{t=0,\ldots,T-1}$
  | Y | Float[Array, 'T'] | $\{y_t\}_{t=0,\ldots,T-1}$

  *$\!$*

In [ ]:
#| hide
@partial(jax.jit, static_argnames=['N', 'T'])
def generate(
    key: PRNGKeyArray, # PRNGKeyArray
    N: int, # $N$
    T: int, # $T$
    p: Parameter # Parameter
) -> WXY:
  key_w, key_xy = jrd.split(key, 2)
  W = gen00.gen_w(key_w, N, T, p.G, p.w0)
  X, Y = gen00.gen_xy(key_xy, N, T, p.S, W)
  return WXY(W, X, Y)

In [ ]:
#| echo: false
sp.reshow_doc(generate)

---

### generate

>      generate (key:Union[jaxKey[Array,''],jaxUInt32[Array,'2']], N:int, T:int,
>                p:__main__.Parameter)

|$\!$| **Type** | **Details** |
| -- | -------- | ----------- |
| key | Union | PRNGKeyArray |
| N | int | $N$ |
| T | int | $T$ |
| p | Parameter | Parameter |
| **Returns** | **WXY** |$\!$|

In [ ]:
#| hide
def generate_main(
    param: Param, # Param
    seed: int # seed値
):
  r"""$\!$*
  データを生成し、00_Data.h5 に保存する。
  *$\!$"""
  N, T, p = restore_param(param)
  batched_generate = jax.vmap(
    lambda key: generate(key, N, T, p)
  )
  master_key = jrd.PRNGKey(seed)
  keys = jrd.split(master_key, 1000)
  wxy = batched_generate(keys)
  save_data(param, "Gen", wxy._asdict())

In [ ]:
#| echo: false
sp.reshow_doc(generate_main)

---

### generate_main

>      generate_main (param:__main__.Param, seed:int)

*$\!$*
データを生成し、00_Data.h5 に保存する。
*$\!$*

|$\!$| **Type** | **Details** |
| -- | -------- | ----------- |
| param | Param | Param |
| seed | int | seed値 |

In [ ]:
#| hide
def predict_main(
    param: Param, # Param
    model_name: str, # EKF, etc.
    func: Callable[[int, int, Float[Array, "T N"], Float[Array, "T"], Parameter], NamedTuple] # $N,T,\{\mathbf x_t\}_{t=0,\ldots,T-1},\{y_t\}_{t=0,\ldots,T-1},\mathrm{p}\to\{\hat{\mathbf w_t}\}_{t=0,\ldots,T-1},\{\mathbf P_{t/t}\}_{t=0,\ldots,T-1},\ldots$
):
  r"""$\!$*
  00_Data.h5 のデータ $X,Y$ に対して `func` で $W$ 等を推論し、保存する。
  *$\!$"""
  N, T, p = restore_param(param)
  batched_predict = jax.vmap(
    lambda X, Y: func(N, T, X, Y, p)
  )
  with h5py.File("00_Data.h5", mode="r") as f:
    dirparam = f"/p{param.N}{param.p}{param.q}{param.r}/Gen"
    groupparam = f.require_group(name=dirparam)
    X = groupparam.require_dataset(name="X", shape=(1000, T, N), dtype=jnp.float32)[...]
    Y = groupparam.require_dataset(name="Y", shape=(1000, T), dtype=jnp.float32)[...]
  WP = batched_predict(X, Y)
  save_data(param, model_name, WP._asdict())

In [ ]:
#| echo: false
sp.reshow_doc(predict_main)

---

### predict_main

>      predict_main (param:__main__.Param, model_name:str, func:Callable[[int,in
>                    t,jaxFloat[Array,'TN'],jaxFloat[Array,'T'],__main__.Paramet
>                    er],NamedTuple])

*$\!$*
00_Data.h5 のデータ $X,Y$ に対して `func` で $W$ 等を推論し、保存する。
*$\!$*

|$\!$| **Type** | **Details** |
| -- | -------- | ----------- |
| param | Param | Param |
| model_name | str | EKF, etc. |
| func | Callable | $N,T,\{\mathbf x_t\}_{t=0,\ldots,T-1},\{y_t\}_{t=0,\ldots,T-1},\mathrm{p}\to\{\hat{\mathbf w_t}\}_{t=0,\ldots,T-1},\{\mathbf P_{t/t}\}_{t=0,\ldots,T-1},\ldots$ |

## 関数のテスト

In [ ]:
#| eval: false
generate_main(Param(2, -3, -3, 0), 0)

In [ ]:
#| eval: false
predict_main(
  Param(2, -3, -3, 0), 
  "EKF", 
  lambda N, T, x, y, p: EKF.EKF(N, T, x, y, p.G, p.w0, p.P0)
)

In [ ]:
#| eval: false
predict_main(
  Param(2, -3, -3, 0), 
  "VApre", 
  lambda N, T, x, y, p: VA.VApre(N, T, x, y, p.G, p.w0, p.P0)
)

In [ ]:
#| eval: false
predict_main(
  Param(2, -3, -3, 0), 
  "VAEM", 
  lambda N, T, x, y, p: VA.VAEM(N, T, x, y, p.G, p.w0, p.P0, p.epsilon)
)

In [ ]:
#| eval: false
with h5py.File("00_Data.h5", mode="w") as f:
    param = Param(2, -3, -3, 0)
    try:
        del f[f"/p{param.N}{param.p}{param.q}{param.r}"]
    except:
        pass

## データ生成

In [ ]:
#| eval: false
SEED = 0
Ns = [2, 4, 8]
ps = jnp.array([-4, -6, -8, -10, -12], dtype=jnp.int32)
qs = jnp.array([-2, -1, 0, 1, 2, 3, 4], dtype=jnp.int32)
rs = jnp.array([0, 1, 2], dtype=jnp.int32)
func_dict: dict[str, Callable[[int, int, Float[Array, "T N"], Float[Array, "T"], Parameter], NamedTuple]] = {
  "EKF": lambda N, T, x, y, p: EKF.EKF(N, T, x, y, p.G, p.w0, p.P0),
  "VApre": lambda N, T, x, y, p: VA.VApre(N, T, x, y, p.G, p.w0, p.P0),
  "VAEM": lambda N, T, x, y, p: VA.VAEM(N, T, x, y, p.G, p.w0, p.P0, p.epsilon)
}

In [ ]:
#| eval: false
count = 0
total = len(ps) * len(qs) * len(rs) * len(Ns) * len(func_dict)
for p in ps:
  for q in qs:
    for r in rs:
      for N in Ns:
        param = Param(N, p, q, r)
        generate_main(param, SEED)
        for n, f in func_dict.items():
          count += 1
          print(f"[{count}/{total}] p={p}, q={q}, r={r}, N={N}, func={n}")
          predict_main(param, n, f)